In [1]:
pip install psycopg2

Note: you may need to restart the kernel to use updated packages.


In [2]:
import psycopg2
import pandas as pd
from IPython.display import Math
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn import tree

In [3]:
list_client = []
list_credit = []
entropias = {}

In [4]:
conn = psycopg2.connect(database = "postgres", 
                        user = "postgres", 
                        host= 'localhost',
                        password = "1234",
                        port = 5432)

In [5]:
query='select * from credit_record'
cur = conn.cursor()
cur.execute(query)
rows = cur.fetchall()
conn.commit()
for row in rows:
    list_credit.append(row)

In [6]:
query='select * from application_record'
cur = conn.cursor()
cur.execute(query)
rows = cur.fetchall()
conn.commit()
conn.close()
for row in rows:
    list_client.append(row)

In [7]:
# Carrega os dados como dataframe do Pandas
df_client = pd.DataFrame(list_client, columns = ['ID_CLIENT','CODE_GENDER','FLAG_OWN_CAR','FLAG_OWN_REALTY','CNT_CHILDREN','AMT_INCOME_TOTAL','NAME_INCOME_TYPE','NAME_EDUCATION_TYPE','NAME_FAMILY_STATUS','NAME_HOUSING_TYPE','DAYS_BIRTH','DAYS_EMPLOYED','FLAG_MOBIL','FLAG_WORK_PHONE','FLAG_PHONE','FLAG_EMAIL','OCCUPATION_TYPE','CNT_FAM_MEMBERS','ID'])
df_credit = pd.DataFrame(list_credit, columns = ['ID_CLIENT','MONTHS_BALANCE','STATUS','ID'])

In [8]:
df_credit.head()

,ID_CLIENT,MONTHS_BALANCE,STATUS,ID
0,5001711,0,X,1
1,5001711,-1,0,2
2,5001711,-2,0,3
3,5001711,-3,0,4
4,5001712,0,C,5


In [9]:
df_client.head()

,ID_CLIENT,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,ID
0,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,None,2.0,1
1,5008805,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,None,2.0,2
2,5008806,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.0,3
3,5008808,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0,4
4,5008809,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0,5


In [10]:
## Validações iniciais das bases e tratamentos - Início

In [11]:
df_client.shape

(438557, 19)

In [12]:
df_credit.shape

(1048575, 4)

In [13]:
df_client.info

<bound method DataFrame.info of         ID_CLIENT CODE_GENDER FLAG_OWN_CAR FLAG_OWN_REALTY CNT_CHILDREN  \
0         5008804           M            Y               Y            0   
1         5008805           M            Y               Y            0   
2         5008806           M            Y               Y            0   
3         5008808           F            N               Y            0   
4         5008809           F            N               Y            0   
...           ...         ...          ...             ...          ...   
438552    6840104           M            N               Y            0   
438553    6840222           F            N               N            0   
438554    6841878           F            N               N            0   
438555    6842765           F            N               Y            0   
438556    6842885           F            N               Y            0   

       AMT_INCOME_TOTAL      NAME_INCOME_TYPE            NAME_EDUCA

In [14]:
df_credit.info

<bound method DataFrame.info of          ID_CLIENT MONTHS_BALANCE STATUS       ID
0          5001711              0      X        1
1          5001711             -1      0        2
2          5001711             -2      0        3
3          5001711             -3      0        4
4          5001712              0      C        5
...            ...            ...    ...      ...
1048570    5150487            -25      C  1048571
1048571    5150487            -26      C  1048572
1048572    5150487            -27      C  1048573
1048573    5150487            -28      C  1048574
1048574    5150487            -29      C  1048575

[1048575 rows x 4 columns]>

In [15]:
# Verifica se há valores ausentes
df_credit.isnull().sum()

ID_CLIENT         0
MONTHS_BALANCE    0
STATUS            0
ID                0
dtype: int64

In [16]:
df_client.isnull().sum()

ID_CLIENT                   0
CODE_GENDER                 0
FLAG_OWN_CAR                0
FLAG_OWN_REALTY             0
CNT_CHILDREN                0
AMT_INCOME_TOTAL            0
NAME_INCOME_TYPE            0
NAME_EDUCATION_TYPE         0
NAME_FAMILY_STATUS          0
NAME_HOUSING_TYPE           0
DAYS_BIRTH                  0
DAYS_EMPLOYED               0
FLAG_MOBIL                  0
FLAG_WORK_PHONE             0
FLAG_PHONE                  0
FLAG_EMAIL                  0
OCCUPATION_TYPE        134203
CNT_FAM_MEMBERS             0
ID                          0
dtype: int64

In [17]:
#Ajustando os valores vazios de OCCUPATION_TYPE para: Not Defined
df_client = df_client.fillna('Not Defined')

In [18]:
# Fazendo o merge dos dataframes usando a coluna 'id'
df_geral = pd.merge(df_client, df_credit, on='ID_CLIENT')
df_geral.head()

,ID_CLIENT,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,...,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,ID_x,MONTHS_BALANCE,STATUS,ID_y
0,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,...,1,1,0,0,Not Defined,2.0,1,0,C,92942
1,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,...,1,1,0,0,Not Defined,2.0,1,-1,C,92943
2,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,...,1,1,0,0,Not Defined,2.0,1,-2,C,92944
3,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,...,1,1,0,0,Not Defined,2.0,1,-3,C,92945
4,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,...,1,1,0,0,Not Defined,2.0,1,-4,C,92946


In [19]:
#Verificar os tipos de cada coluna
for coluna in df_geral.columns:
    print(df_geral[coluna].info())

<class 'pandas.core.series.Series'>
RangeIndex: 777715 entries, 0 to 777714
Series name: ID_CLIENT
Non-Null Count   Dtype
--------------   -----
777715 non-null  int64
dtypes: int64(1)
memory usage: 5.9 MB
None
<class 'pandas.core.series.Series'>
RangeIndex: 777715 entries, 0 to 777714
Series name: CODE_GENDER
Non-Null Count   Dtype 
--------------   ----- 
777715 non-null  object
dtypes: object(1)
memory usage: 5.9+ MB
None
<class 'pandas.core.series.Series'>
RangeIndex: 777715 entries, 0 to 777714
Series name: FLAG_OWN_CAR
Non-Null Count   Dtype 
--------------   ----- 
777715 non-null  object
dtypes: object(1)
memory usage: 5.9+ MB
None
<class 'pandas.core.series.Series'>
RangeIndex: 777715 entries, 0 to 777714
Series name: FLAG_OWN_REALTY
Non-Null Count   Dtype 
--------------   ----- 
777715 non-null  object
dtypes: object(1)
memory usage: 5.9+ MB
None
<class 'pandas.core.series.Series'>
RangeIndex: 777715 entries, 0 to 777714
Series name: CNT_CHILDREN
Non-Null Count   Dtype 
----

In [20]:
#Verifica soma de valores unicos de cada coluna
for coluna in df_geral.columns:
    print(df_geral[coluna].value_counts())

ID_CLIENT
5090630    61
5148524    61
5066707    61
5061848    61
5118380    61
           ..
5024557     1
5062311     1
5024365     1
5024364     1
5041568     1
Name: count, Length: 36457, dtype: int64
CODE_GENDER
F    518851
M    258864
Name: count, dtype: int64
FLAG_OWN_CAR
N    473355
Y    304360
Name: count, dtype: int64
FLAG_OWN_REALTY
Y    512948
N    264767
Name: count, dtype: int64
CNT_CHILDREN
0     540639
1     155638
2      70399
3       9328
4       1224
5        324
14       111
7         46
19         6
Name: count, dtype: int64
AMT_INCOME_TOTAL
135000.0    90217
180000.0    68579
157500.0    62686
112500.0    61622
225000.0    61399
            ...  
51750.0         7
89550.0         5
594000.0        4
151200.0        2
134995.5        2
Name: count, Length: 265, dtype: int64
NAME_INCOME_TYPE
Working                 400164
Commercial associate    183385
Pensioner               128392
State servant            65437
Student                    337
Name: count, dtype: in

In [21]:
## Validações iniciais das bases e tratamentos - Fim

In [22]:
## Calculando Entropias - Início

In [23]:
Math(r'Y = {-\sum \limits _{i=1} ^{N} p(y_{i})*log2(p(y_{i}))}')

<IPython.core.display.Math object>

In [24]:
## Onde
## - Y é a variável alvo
## - N é o número total de classes diferentes na variável.
## - p(y)  é a proporção (ou probabilidade) da classe i no conjunto de dados

In [26]:
# Formula da entropia
def entropia(serie):
    # Contar a frequência de cada valor (classe) e normalizar para obter proporções (probabilidades)
    frequencias = serie.value_counts(normalize=True)
    # Calcular a entropia usando a fórmula
    entropia = -np.sum(frequencias * np.log2(frequencias))
    return entropia

# Função para calcular a entropia de uma série (coluna) do dataframe
def calcular_entropia(df):
    for coluna in df.columns:
        # Verificando se as colunas são categóricas ou numéricas e calculando a entropia
        if df[coluna].dtype == 'object' or df[coluna].nunique() < 15:  # Para categóricas ou numéricas discretas
            entropias[coluna] = entropia(df[coluna])
        else:
            # Para colunas numéricas, primeiro discretizamos (agrupamos em bins)
            bin_coluna = coluna + '_bins'
            df[bin_coluna] = pd.qcut(df[coluna], q=4, duplicates='drop')  # Aqui estamos usando 4 bins (quartis)
            entropias[coluna] = entropia(df[bin_coluna])
            df.drop(columns=[bin_coluna], inplace=True)  # Remover a coluna de bins após o cálculo
            
# Aplicar a função aos dataframes
calcular_entropia(df_client)
#calcular_entropia_df(df_client)

# Exibir o resultado
for coluna, entropia in entropias.items():
    print(f"Entropia da coluna {coluna}: {entropia}")


Entropia da coluna ID_CLIENT: 1.9999999999887483
Entropia da coluna MONTHS_BALANCE: 5.614742606398652
Entropia da coluna STATUS: 1.619069103206018
Entropia da coluna ID: 1.9999999999887483
Entropia da coluna CODE_GENDER: 0.9135064524335109
Entropia da coluna FLAG_OWN_CAR: 0.9521174750328749
Entropia da coluna FLAG_OWN_REALTY: 0.889267184391278
Entropia da coluna CNT_CHILDREN: 1.2407460822877812
Entropia da coluna AMT_INCOME_TOTAL: 5.124581172661348
Entropia da coluna NAME_INCOME_TYPE: 1.714758860037736
Entropia da coluna NAME_EDUCATION_TYPE: 1.115362690289542
Entropia da coluna NAME_FAMILY_STATUS: 1.5003380561934658
Entropia da coluna NAME_HOUSING_TYPE: 0.6703530934426803
Entropia da coluna DAYS_BIRTH: 13.711215123227127
Entropia da coluna DAYS_EMPLOYED: 10.987988113528237
Entropia da coluna FLAG_MOBIL: -0.0
Entropia da coluna FLAG_WORK_PHONE: 0.7340254844618739
Entropia da coluna FLAG_PHONE: 0.8658245071658033
Entropia da coluna FLAG_EMAIL: 0.4944843626487706
Entropia da coluna OCCUPA

In [27]:
## Calculando Entropias - Fim

In [28]:
## Calculando Ganho de Informação - Início

In [29]:
# Verifique e converta a coluna AMT_INCOME_TOTAL para numérico, forçando erros para NaN
df_geral['AMT_INCOME_TOTAL'] = pd.to_numeric(df_geral['AMT_INCOME_TOTAL'], errors='coerce')

# Verifique se há valores NaN após a conversão
print(df_geral['AMT_INCOME_TOTAL'].isna().sum(), "valores NaN encontrados")

# Opcional: remover ou tratar valores NaN antes de continuar
df_geral.dropna(subset=['AMT_INCOME_TOTAL'], inplace=True)

# Agora, discretize a variável numérica em quartis
df_geral['AMT_INCOME_BIN'] = pd.qcut(df_geral['AMT_INCOME_TOTAL'], q=4, labels=False)  # 4 faixas (quartis)


0 valores NaN encontrados


In [ ]:
#Não executar!

df_geral_sample = df_geral.sample(frac=0.2, random_state=1)  # 20% dos dados

# Inicializando o dicionário de ganhos de informação
gi = {}

# Função para calcular a entropia de uma série (coluna)
def calcular_entropia(serie):
    frequencias = serie.value_counts(normalize=True)
    entropia = -np.sum(frequencias * np.log2(frequencias))
    return entropia

# Função para calcular o ganho de informação utilizando as entropias já calculadas
def ganho_informacao(df, target, entropias):
    for coluna in df.columns:
        if coluna != target:  # Não aplicar o cálculo para a coluna-alvo (STATUS)
            print(f"Processando coluna: {coluna}")
            
            # Entropia inicial da coluna-alvo (STATUS) já calculada no dicionário
            entropia_inicial = entropias[target]
            print(f"Entropia inicial (STATUS): {entropia_inicial}")

            # Entropia condicional após dividir pelos valores da feature
            valores_unicos = df[coluna].unique()
            entropia_condicional = 0
            for valor in valores_unicos:
                subconjunto = df[df[coluna] == valor]
                
                # Verificar se há valores nulos na coluna-alvo para esse subconjunto
                if subconjunto[target].isnull().any():
                    print(f"Subconjunto ignorado devido a valores nulos na coluna-alvo {target}")
                    continue  # Ignorar subconjunto se houver nulos na coluna-alvo

                proporcao = len(subconjunto) / len(df)
                
                # Calcular a entropia da coluna 'target' no subconjunto
                entropia_subconjunto = calcular_entropia(subconjunto[target])
                
                # Calcular a entropia condicional acumulando proporcionalmente a cada subconjunto
                entropia_condicional += proporcao * entropia_subconjunto

            # Calcular o ganho de informação
            ganho = entropia_inicial - entropia_condicional
            gi[coluna] = ganho
            print(f"Ganho de Informação da coluna {coluna}: {ganho}")

# Aplicando o ganho de informação para o dataframe e a variável-alvo 'STATUS'
ganho_informacao(df_geral_sample, 'STATUS', entropias)

# Exibir os resultados
if gi:
    for coluna, ganho in gi.items():
        print(f"Ganho de Informação da coluna {coluna}: {ganho}")
else:
    print("Nenhum ganho de informação foi calculado.")


In [30]:
# Função para calcular a entropia de uma série (coluna)
def calcular_entropia(serie):
    frequencias = serie.value_counts(normalize=True)
    entropia = -np.sum(frequencias * np.log2(frequencias))
    return entropia

# Função para calcular o ganho de informação
def ganho_informacao(df, feature, target):
    # Entropia antes de dividir
    entropia_inicial = calcular_entropia(df[target])

    # Entropia após dividir pelos valores da feature
    valores_unicos = df[feature].unique()
    entropia_condicional = 0
    for valor in valores_unicos:
        subconjunto = df[df[feature] == valor]
        proporcao = len(subconjunto) / len(df)
        entropia_condicional += proporcao * calcular_entropia(subconjunto[target])

    # Ganho de informação
    ganho = entropia_inicial - entropia_condicional
    return ganho

# Aplicando o ganho de informação após o merge
gi_NAME_INCOME_TYPE = ganho_informacao(df_geral, 'NAME_INCOME_TYPE', 'STATUS')
gi_AMT_INCOME_TOTAL = ganho_informacao(df_geral, 'AMT_INCOME_BIN', 'STATUS')  # Use a versão discretizada!
gi_OCCUPATION_TYPE = ganho_informacao(df_geral, 'OCCUPATION_TYPE', 'STATUS')
gi_NAME_EDUCATION_TYPE = ganho_informacao(df_geral, 'NAME_EDUCATION_TYPE', 'STATUS')
gi_NAME_HOUSING_TYPE = ganho_informacao(df_geral, 'NAME_HOUSING_TYPE', 'STATUS')
gi_CNT_FAM_MEMBERS = ganho_informacao(df_geral, 'CNT_FAM_MEMBERS', 'STATUS')
gi_FLAG_OWN_CAR = ganho_informacao(df_geral, 'FLAG_OWN_CAR', 'STATUS')
gi_FLAG_OWN_REALTY = ganho_informacao(df_geral, 'FLAG_OWN_REALTY', 'STATUS')
gi_CNT_CHILDREN = ganho_informacao(df_geral, 'CNT_CHILDREN', 'STATUS')

print(f"Ganho de Informação para a STATUS 'NAME_INCOME_TYPE': {gi_NAME_INCOME_TYPE}")
print(f"Ganho de Informação para a STATUS 'AMT_INCOME_TOTAL' (discretizado): {gi_AMT_INCOME_TOTAL}")
print(f"Ganho de Informação para a STATUS 'OCCUPATION_TYPE': {gi_OCCUPATION_TYPE}")
print(f"Ganho de Informação para a STATUS 'NAME_EDUCATION_TYPE': {gi_OCCUPATION_TYPE}")
print(f"Ganho de Informação para a STATUS 'NAME_HOUSING_TYPE': {gi_NAME_HOUSING_TYPE}")
print(f"Ganho de Informação para a STATUS 'CNT_FAM_MEMBERS': {gi_CNT_FAM_MEMBERS}")
print(f"Ganho de Informação para a STATUS 'NAME_EDUCATION_TYPE': {gi_NAME_EDUCATION_TYPE}")
print(f"Ganho de Informação para a STATUS 'FLAG_OWN_CAR': {gi_FLAG_OWN_CAR}")
print(f"Ganho de Informação para a STATUS 'FLAG_OWN_REALTY': {gi_FLAG_OWN_REALTY}")
print(f"Ganho de Informação para a STATUS 'CNT_CHILDREN': {gi_CNT_CHILDREN}")

Ganho de Informação para a STATUS 'NAME_INCOME_TYPE': 0.0004885641362688542
Ganho de Informação para a STATUS 'AMT_INCOME_TOTAL' (discretizado): 0.0007124961086473203
Ganho de Informação para a STATUS 'OCCUPATION_TYPE': 0.0022337938053160755
Ganho de Informação para a STATUS 'NAME_EDUCATION_TYPE': 0.0022337938053160755
Ganho de Informação para a STATUS 'NAME_HOUSING_TYPE': 0.00045854203801343196
Ganho de Informação para a STATUS 'CNT_FAM_MEMBERS': 0.0008715322924137681
Ganho de Informação para a STATUS 'NAME_EDUCATION_TYPE': 0.0009402311038140265
Ganho de Informação para a STATUS 'FLAG_OWN_CAR': 0.0001213543315026211
Ganho de Informação para a STATUS 'FLAG_OWN_REALTY': 0.00014566089061651688
Ganho de Informação para a STATUS 'CNT_CHILDREN': 0.0006715875510958647


In [ ]:
## Ganho da Informação - Fim

In [ ]:
## Exploração de dados - Início

In [32]:
# Verifique e converta a coluna AMT_INCOME_TOTAL para numérico, forçando erros para NaN
df_geral['CNT_FAM_MEMBERS'] = pd.to_numeric(df_geral['CNT_FAM_MEMBERS'], errors='coerce')

# Verifique se há valores NaN após a conversão
print(df_geral['CNT_FAM_MEMBERS'].isna().sum(), "valores NaN encontrados")


0 valores NaN encontrados


In [45]:
#Renda por pessoa
df_geral['INCOME_PER_FAMILY_MEMBER'] = df_geral['AMT_INCOME_TOTAL'] / df_geral['CNT_FAM_MEMBERS']

In [37]:
renda_media_por_ocupacao = df_geral.groupby('OCCUPATION_TYPE')['AMT_INCOME_TOTAL'].transform('mean')
df_geral['INCOME_VS_OCCUPATION_AVG'] = df_geral['AMT_INCOME_TOTAL'] / renda_media_por_ocupacao

In [43]:
df_geral['INCOME_EDUCATION_COMBO'] = df_geral['NAME_INCOME_TYPE'] + "_" + df_geral['NAME_EDUCATION_TYPE']

,ID_CLIENT,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,...,CNT_FAM_MEMBERS,ID_x,MONTHS_BALANCE,STATUS,ID_y,AMT_INCOME_BIN,INCOME_PER_FAMILY_MEMBER,INCOME_OCCUPATION,INCOME_VS_OCCUPATION_AVG,INCOME_EDUCATION_COMBO
0,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,...,2.0,1,0,C,92942,3,213750.0,427500.0_Not Defined,2.524927,Working_Higher education
1,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,...,2.0,1,-1,C,92943,3,213750.0,427500.0_Not Defined,2.524927,Working_Higher education
2,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,...,2.0,1,-2,C,92944,3,213750.0,427500.0_Not Defined,2.524927,Working_Higher education
3,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,...,2.0,1,-3,C,92945,3,213750.0,427500.0_Not Defined,2.524927,Working_Higher education
4,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,...,2.0,1,-4,C,92946,3,213750.0,427500.0_Not Defined,2.524927,Working_Higher education
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,5008810,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,...,1.0,6,-24,0,93037,3,270000.0,270000.0_Sales staff,1.543930,Commercial associate_Secondary / secondary spe...
96,5008810,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,...,1.0,6,-25,X,93038,3,270000.0,270000.0_Sales staff,1.543930,Commercial associate_Secondary / secondary spe...
97,5008810,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,...,1.0,6,-26,X,93039,3,270000.0,270000.0_Sales staff,1.543930,Commercial associate_Secondary / secondary spe...
98,5008811,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,...,1.0,7,0,C,93040,3,270000.0,270000.0_Sales staff,1.543930,Commercial associate_Secondary / secondary spe...


In [44]:
for coluna in ['INCOME_PER_FAMILY_MEMBER', 'INCOME_VS_OCCUPATION_AVG', 'INCOME_EDUCATION_COMBO']:
    ganho = ganho_informacao(df_geral, coluna, 'STATUS')
    print(f"Ganho de Informação para {coluna}: {ganho}")


Ganho de Informação para INCOME_PER_FAMILY_MEMBER: 0.02868921238225397
Ganho de Informação para INCOME_VS_OCCUPATION_AVG: 0.07286605470746421
Ganho de Informação para INCOME_EDUCATION_COMBO: 0.002333481446844532
